Your task is to implement SPOS + Random Search to find the optimal ResNet-18-like architecture
for image classification on CIFAR-10 dataset.

### Proposed search space
The same search space as in the lecture, but without kernel size search.
This means that each search block should only have 3 operations.

### Latency awareness
While searching, you should implement hard constraint on model latency. As a proxy for
latency, use the number of MACs. The baseline ResNet-18 has 37M MACs for images of size 32x32.
The final selected architecture should have at most 30M MACs.
You can find an example computation for the number of MACs in this notebook.

### How to complete the task
Plan for your experiments:
1. Train baseline ResNet-18 model.  **(2 pts)**
2. Finalize supernet code. **(15 pts)**
3. Train supernet with the same hyperparameters except the number of epochs. You should increase
   it by a factor of 3-6. Save supernet weights.  **(10 pts)**
4. Write implementation for Random Search.  **(10 pts)**
5. Run it for 100-1000 iterations by measuring accuracies of models from the supernet.
   Keep track of each model accuracy and latency.  **(5 pts)**
6. Train the best architecture from scratch. For this, you can build the whole supernet, and
   select this architecture for all the forward passes of the training.  **(6 pts)**
7. Compare with the model from step 1. Does your model have a better accuracy?  **(2 pts)**

### Code structure

Main.ipynb - Learn your neural networks here.

resnet.py - ResNet-18 implementation from torchvision, simplified for this project.

supernet.py - Unfinished Supernet implementation.

cifar10.py - Transforms for CIFAR-10 training.


In [1]:
!pip install torch torchvision numpy thop tqdm

Defaulting to user installation because normal site-packages is not writeable


### Imports

In [2]:
from typing import Tuple

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from thop import profile
from torch.utils.data import DataLoader
from tqdm import tqdm

from cifar10 import get_train_transform, get_val_transform
from resnet import resnet18

### Make everything a bit faster

In [3]:
torch.backends.cudnn.benchmark = True
torch.backends.cudnn.deterministic = False

In [4]:
# Change this value if needed.
batch_size = 512

In [5]:
train_transform = get_train_transform()
val_transform = get_val_transform()

train_set = torchvision.datasets.CIFAR10(
    root='./data',
    train=True,
    download=True,
    transform=train_transform,
)
test_set = torchvision.datasets.CIFAR10(
    root='./data',
    train=False,
    download=True,
    transform=val_transform,
)

train_dataloader = DataLoader(
    train_set,
    batch_size=batch_size,
    shuffle=True,
    num_workers=4,
    drop_last=True,
)
test_dataloader = DataLoader(
    test_set,
    batch_size=batch_size,
    shuffle=False,
    num_workers=4,
    drop_last=False,
)

Files already downloaded and verified
Files already downloaded and verified


## Create your baseline model

In [6]:
# Select suitable device.
# You should probably use either cuda (NVidia GPU) or mps (Apple) backend.
device = torch.device('cuda:0')
model = resnet18(num_classes=10, zero_init_residual=True)
model.to(device=device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

### Compute the number of MACs and parameters for the model

In [7]:
macs, params = profile(model, inputs=(torch.zeros(1, 3, 32, 32, device=device),))
print(f'Number of macs: {macs / 1e6:.2f}M, number of parameters: {params / 1e6:.2f}M')

[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.batchnorm.BatchNorm2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.pooling.MaxPool2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.container.Sequential'>.
[INFO] Register count_adap_avgpool() for <class 'torch.nn.modules.pooling.AdaptiveAvgPool2d'>.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
Number of macs: 37.22M, number of parameters: 11.18M


## Train baseline model

### Define loss function

In [8]:
criterion = nn.CrossEntropyLoss()

### Select hyperparameters

In [10]:
lr = 0.25
weight_decay = 5e-4
momentum = 0.9
n_epochs = 20  # Longer training gives better results, but let's keep baseline model epochs to 20.

### Build optimizer and scheduler

In [11]:
optimizer = optim.SGD(model.parameters(), lr=lr, weight_decay=weight_decay, momentum=momentum)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=len(train_dataloader) * n_epochs)

### Define training and evaluation functions

In [12]:
def train_one_epoch(
        model: nn.Module,
        criterion: nn.Module,
        dataloader: DataLoader,
        optimizer: optim.Optimizer,
        scheduler,
        device: torch.device,
        epoch: int,
) -> Tuple[float, float]:
    model.train()

    total_loss = 0.0
    total_correct = 0.0
    total_samples = 0

    wrapped_dataloader = tqdm(enumerate(dataloader), total=len(dataloader))
    for i, (inputs, labels) in wrapped_dataloader:
        inputs = inputs.to(device=device)
        labels = labels.to(device=device)

        optimizer.zero_grad()

        logits = model(inputs)
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()
        scheduler.step()

        with torch.no_grad():
            _, predicted_labels = torch.max(logits, 1)
            total_loss += loss.item()
            total_correct += (predicted_labels == labels).sum().item()
            total_samples += labels.shape[0]

        wrapped_dataloader.set_description(
            f'(train) Epoch={epoch}, lr={scheduler.get_last_lr()[0]:.4f} loss={total_loss / (i + 1):.3f}'
        )

    return total_loss / len(dataloader), total_correct / total_samples


@torch.no_grad()
def validate_one_epoch(
        model: nn.Module,
        criterion: nn.Module,
        dataloader: DataLoader,
        device: torch.device,
        epoch: int,
) -> Tuple[float, float]:
    model.eval()

    total_loss = 0.0
    total_correct = 0.0
    total_samples = 0

    wrapped_dataloader = tqdm(enumerate(dataloader), total=len(dataloader))
    for i, (inputs, labels) in wrapped_dataloader:
        inputs = inputs.to(device=device)
        labels = labels.to(device=device)

        logits = model(inputs)
        loss = criterion(logits, labels)
        _, predicted_labels = torch.max(logits, 1)
        total_loss += loss.item()
        total_correct += (predicted_labels == labels).sum().item()
        total_samples += labels.shape[0]

        wrapped_dataloader.set_description(f'(val) Epoch={epoch}, loss={total_loss / (i + 1):.3f}')

    return total_loss / len(dataloader), total_correct / total_samples

### Run training

In [13]:
for epoch in range(n_epochs):
    print(f'Epoch: {epoch}')
    loss, accuracy = train_one_epoch(model, criterion, train_dataloader, optimizer, scheduler, device, epoch)
    print(f'train_loss={loss:.4f}, train_accuracy={accuracy:.3%}')
    loss, accuracy = validate_one_epoch(model, criterion, test_dataloader, device, epoch)
    print(f'test_loss={loss:.4f}, test_accuracy={accuracy:.3%}')

torch.save(model.state_dict(), 'baseline_model.pth')

Epoch: 0


(train) Epoch=0, lr=0.2485 loss=2.011: 100%|██████████| 97/97 [00:03<00:00, 31.91it/s]

train_loss=2.0107, train_accuracy=28.328%



(val) Epoch=0, loss=1.589: 100%|██████████| 20/20 [00:00<00:00, 49.33it/s]

test_loss=1.5893, test_accuracy=38.970%
Epoch: 1


...

test_loss=0.5090, test_accuracy=82.660%
Epoch: 19



(train) Epoch=19, lr=0.0000 loss=0.397: 100%|██████████| 97/97 [00:03<00:00, 30.53it/s]

train_loss=0.3969, train_accuracy=86.203%



(val) Epoch=19, loss=0.508: 100%|██████████| 20/20 [00:00<00:00, 62.18it/s]


test_loss=0.5080, test_accuracy=82.780%
